A good tool makes simple things easy and complex things possible.

If you don't know which solution is better, start with the one that's simpler.

In [1]:
import os
from yaml import load, CLoader as Loader

data = load(open('env/local-env.yml'), Loader=Loader)
os.environ.update({key: str(val) for key, val in data.items()})

from hemlock import push_app_context

app = push_app_context()

In [2]:
from hemlock import Participant, Embedded

part = Participant.gen_test_participant()

In [3]:
emb1 = Embedded('var', 1)
emb0 = Embedded('var', 0)
part.embedded = [emb0, emb1]
dict(part.get_data())

[<Embedded 14>, <Embedded 13>]
setting order for <Embedded 14>
data is 0
order is 0
setting order for <Embedded 13>
data is 1
order is 1


{'ID': [4, 4],
 'EndTime': [datetime.datetime(2020, 7, 11, 18, 24, 53, 263746),
  datetime.datetime(2020, 7, 11, 18, 24, 53, 263746)],
 'StartTime': [datetime.datetime(2020, 7, 11, 18, 24, 53, 263746),
  datetime.datetime(2020, 7, 11, 18, 24, 53, 263746)],
 'Status': ['InProgress', 'InProgress'],
 'var': [1, 0],
 'varOrder': [1, 0],
 'varIndex': [1, 0]}